# $\Delta$ITD and $\Delta$ILD

Sourced from `https://github.com/vb000/SemanticHearing/blob/main/src/helpers/eval_utils.py`

In [ ]:
import os
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import rfft, irfft
from scipy.signal import stft
from sklearn.utils.extmath import weighted_mode
import torch
import soundfile as sf

In [ ]:
# Constants
SAMPLE_RATE = 44100
ENERGY_THRESHOLD = 5e-4
TMAX = int(1e-3 * SAMPLE_RATE)  # maximum lag in samples = +/- 1 ms

In [ ]:
def tdoa(x1, x2, interp=1, fs=44100, phat=True, t_max=None):
    """
    This function computes the time difference of arrival (TDOA)
    of the signal at the two microphones. This in turns is used to infer
    the direction of arrival (DOA) of the signal.
    Specifically if s(k) is the signal at the reference microphone and
    s_2(k) at the second microphone, then for signal arriving with DOA
    theta we have
    s_2(k) = s(k - tau)
    with
    tau = fs*d*sin(theta)/c
    where d is the distance between the two microphones and c the speed of sound.
    We recover tau using the Generalized Cross Correlation - Phase Transform (GCC-PHAT)
    method. The reference is
    Knapp, C., & Carter, G. C. (1976). The generalized correlation method for estimation of time delay.
    Parameters
    ----------
    x1 : nd-array
        The signal of the reference microphone
    x2 : nd-array
        The signal of the second microphone
    interp : int, optional (default 1)
        The interpolation value for the cross-correlation, it can
        improve the time resolution (and hence DOA resolution)
    fs : int, optional (default 44100 Hz)
        The sampling frequency of the input signal
    Return
    ------
    theta : float
        the angle of arrival (in radian (I think))
    pwr : float
        the magnitude of the maximum cross correlation coefficient
    delay : float
        the delay between the two microphones (in seconds)
    """
    # zero padded length for the FFT
    n = x1.shape[-1] + x2.shape[-1] - 1
    if n % 2 != 0:
        n += 1

    # Generalized Cross Correlation Phase Transform
    # Used to find the delay between the two microphones
    # up to line 71
    X1 = rfft(np.array(x1, dtype=np.float32), n=n, axis=-1)
    X2 = rfft(np.array(x2, dtype=np.float32), n=n, axis=-1)

    if phat:
        X1 /= np.abs(X1) + 1e-15  # added epsilon
        X2 /= np.abs(X2) + 1e-15  # added epsilon

    cc = irfft(X1 * np.conj(X2), n=interp * n, axis=-1)

    # alternative: compute phase spectrum first and then normalize
    # R = X1 * np.conj(X2)
    # R_phat = R / (np.abs(R) + 1e-15)
    # cc = irfft(R_phat, n=interp * n, axis=-1)

    # maximum possible delay given distance between microphones
    if t_max is None:
        t_max = n // 2 + 1

    # reorder the cross-correlation coefficients
    cc = np.concatenate((cc[..., -t_max:], cc[..., :t_max]), axis=-1)

    # pick max cross correlation index as delay
    tau = np.argmax(np.abs(cc), axis=-1)
    tau -= t_max  # because zero time is at the center of the array

    return tau / (fs * interp)

In [ ]:
def compute_ild(s_left, s_right):
    sum_sq_left = np.sum(s_left ** 2, axis=-1)
    sum_sq_right = np.sum(s_right ** 2, axis=-1)
    return 10 * np.log10(sum_sq_left / sum_sq_right)

In [ ]:
def gcc_phat_diff(s_est, s_gt, sr):
    itd_est = tdoa(s_est[..., 0, :], s_est[..., 1, :], fs=sr, t_max=TMAX)
    itd_gt = tdoa(s_gt[..., 0, :], s_gt[..., 1, :], fs=sr, t_max=TMAX)
    return np.abs(itd_est - itd_gt) * 10 ** 6

In [ ]:
def framewise_gccphat(x, frame_dur, sr, window='tukey'):
    frame_width = int(frame_dur * sr)

    # Total number of frames T
    T = (x.shape[-1]) // frame_width

    # Drop samples to get a multiple of frame size
    if x.shape[-1] % T != 0:
        x = x[..., :(frame_width * T)]
    
    assert x.shape[-1] % T == 0

    # Split into frames
    frames = np.array(np.split(x, T, axis=-1))

    # Apply window
    window = signal.get_window(window, frame_width)
    frames = frames * window

    # Consider only frames that have energy above some threshold (ignore silence)
    frame_energy = np.max(np.mean(frames**2, axis=-1)**0.5, axis=-1)
    mask = frame_energy > ENERGY_THRESHOLD
    frames = frames[mask]
    
    fw_gccphat = tdoa(frames[..., 0, :], frames[..., 1, :], fs=sr, t_max=TMAX)

    itd = weighted_mode(fw_gccphat, frame_energy[mask], axis=-1)[0]

    return itd[0]

In [ ]:
def fw_itd_diff(s_est, s_gt, sr, frame_duration=0.25):
    """
    Computes frame-wise delta ITD
    """
    itd_gt = framewise_gccphat(s_gt, frame_duration, sr) * 1e6
    itd_est = framewise_gccphat(s_est, frame_duration, sr) * 1e6
    return np.abs(itd_est - itd_gt)

In [ ]:
def ild_diff(s_est, s_gt):
    """
    Computes the ILD error between model estimate and ground truth
    input: (*, 2, T), (*, 2, T)
    """
    ild_est = compute_ild(s_est[..., 0, :], s_est[..., 1, :])
    ild_gt = compute_ild(s_gt[..., 0, :], s_gt[..., 1, :])
    return np.abs(ild_est - ild_gt)

### Low Frequency Test

In [ ]:
f0 = 200
sr = 44100
N = sr * 5
time_vector = np.linspace(0, 5, num=N)
channel_1 = np.sin(f0 * time_vector)
delay = int(0.001 * sr)  # 1000 microseconds
channel_2 = np.pad(channel_1, (delay, 0))[:N]
y = np.stack((channel_1, channel_2), axis=0)

In [ ]:
print(f"Delta ITD with 0.25 s frames: {framewise_gccphat(y, 0.25, sr) * 1e6}")
print(f"Delta ITD with 0.5 s frames: {framewise_gccphat(y, 0.5, sr) * 1e6}")
print(f"Delta ITD with 1 s frames: {framewise_gccphat(y, 1, sr) * 1e6}")

## Evaluate

In [ ]:
DATE =  datetime.now().strftime("%Y-%m-%d")
MODEL = 'htdemucs_ft'
EVAL_DIR = "../data/eval/sh_metrics/"

In [ ]:
STEMS = ["drums", "bass", "other", "vocals"]

In [ ]:
FRAME_LENGTH = 0.5  # seconds

### Random Binaural Data

In [ ]:
DATASET = 'random'

# set input and output directories
REFERENCE_DIR = f"../data/binaural_musdb18/{DATASET}/test/"
ESTIMATE_DIR = f"../data/output/{MODEL}/{DATASET}/test/"

In [ ]:
# create the output directory if it does not already exist
print("Creating evaluation directory, if it does not already exist...")
os.makedirs(EVAL_DIR, exist_ok=True)

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
song_list = [f for f in os.listdir(REFERENCE_DIR) if os.path.isdir(os.path.join(REFERENCE_DIR, f))]
print(f"There are {len(song_list)} files in the reference directory.")

In [ ]:
title_list = []
source_list = []
itd_list = []
ild_list = []

print("Beginning to evaluate stems...")
for source in STEMS:
    print(f"\n>>>>{source} <<<<")
    for song in tqdm(song_list):
        
        ref_file = os.path.join(REFERENCE_DIR, song, f"{source}.wav")
        est_file = os.path.join(ESTIMATE_DIR, song, f"{source}.wav")

        # load reference and estimate stems
        y_ref, sr_ref = sf.read(ref_file)
        y_est, sr_est = sf.read(est_file)

        # check sample rates
        assert sr_ref == sr_est

        # calculate Delta ITD
        delta_itd = fw_itd_diff(y_est.T, y_ref.T, sr_ref, FRAME_LENGTH)

        # calculate ILD
        delta_ild = ild_diff(y_est.T, y_ref.T)

        title_list.append(song)
        source_list.append(source)
        itd_list.append(delta_itd)
        ild_list.append(delta_ild)

results_df = pd.DataFrame({"title": title_list, "source": source_list,
                           "diff_ITD": itd_list, "diff_ILD": ild_list})

print("Evaluation complete!")

In [ ]:
results_df[['source', 'diff_ITD']].groupby('source').median()

In [ ]:
# sort
results_df.sort_values(by=['title', 'source'], inplace=True, ignore_index=True)

In [ ]:
save_path = os.path.join(EVAL_DIR, f'itd_ild_{DATE}_{MODEL}_{DATASET}.csv')

results_df.to_csv(save_path, index=False)

### Stereo Data

In [ ]:
DATASET = 'stereo'

# set input and output directories
REFERENCE_DIR = f"../data/musdb18hq/test/"
ESTIMATE_DIR = f"../data/output/{MODEL}/{DATASET}/test/"

In [ ]:
# create the output directory if it does not already exist
print("Creating evaluation directory, if it does not already exist...")
os.makedirs(EVAL_DIR, exist_ok=True)

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
song_list = [f for f in os.listdir(REFERENCE_DIR) if os.path.isdir(os.path.join(REFERENCE_DIR, f))]
print(f"There are {len(song_list)} files in the reference directory.")

In [ ]:
title_list = []
source_list = []
itd_list = []
ild_list = []

print("Beginning to evaluate stems...")
for source in STEMS:
    print(f"\n>>>>{source} <<<<")
    for song in tqdm(song_list):
        
        ref_file = os.path.join(REFERENCE_DIR, song, f"{source}.wav")
        est_file = os.path.join(ESTIMATE_DIR, song, f"{source}.wav")

        # load reference and estimate stems
        y_ref, sr_ref = sf.read(ref_file)
        y_est, sr_est = sf.read(est_file)

        # check sample rates
        assert sr_ref == sr_est == SAMPLE_RATE

        # calculate Delta ITD
        delta_itd = fw_itd_diff(y_est.T, y_ref.T, SAMPLE_RATE, FRAME_LENGTH)

        # calculate ILD
        delta_ild = ild_diff(y_est.T, y_ref.T)

        title_list.append(song)
        source_list.append(source)
        itd_list.append(delta_itd)
        ild_list.append(delta_ild)

results_df = pd.DataFrame({"title": title_list, "source": source_list,
                           "diff_ITD": itd_list, "diff_ILD": ild_list})

print("Evaluation complete!")

In [ ]:
# sort
results_df.sort_values(by=['title', 'source'], inplace=True, ignore_index=True)

In [ ]:
save_path = os.path.join(EVAL_DIR, f'itd_ild_{DATE}_{MODEL}_{DATASET}.csv')

results_df.to_csv(save_path, index=False)